In [1]:
import pandas as pd
from sklearn.metrics import f1_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split

from simpletransformers.classification import ClassificationModel, ClassificationArgs
import torch

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
LEARNING_RATE = 1e-5
EPOCHS = 5
BATCH_SIZE = 16
DS_SPLIT = 0.2
MIN_SPECIALITY_THRESHOLD = 100
DATASET_PATH = 'data/mtsamples.csv'

In [3]:
df = pd.read_csv(DATASET_PATH, usecols=['keywords', 'transcription', 'medical_specialty'])    
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   medical_specialty  4999 non-null   object
 1   transcription      4966 non-null   object
 2   keywords           3931 non-null   object
dtypes: object(3)
memory usage: 117.3+ KB


,medical_specialty,transcription,keywords
0,Allergy / Immunology,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,Bariatrics,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,Bariatrics,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,Cardiovascular / Pulmonary,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,Cardiovascular / Pulmonary,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [4]:
df.dropna(inplace=True)
counts = df['medical_specialty'].value_counts()
others = [k for k,v in counts.items() if v < MIN_SPECIALITY_THRESHOLD]
for each_spec in others:
    df.loc[df['medical_specialty']==each_spec,'medical_specialty']=' others' 

counts = df['medical_specialty'].value_counts()
print(counts)

medical_specialty
Surgery                          1021
others                            888
Orthopedic                        303
Cardiovascular / Pulmonary        280
Radiology                         251
Consult - History and Phy.        234
Gastroenterology                  195
Neurology                         168
General Medicine                  146
SOAP / Chart / Progress Notes     142
Urology                           140
Obstetrics / Gynecology           130
Name: count, dtype: int64


In [5]:
num_classes = len(df['medical_specialty'].unique())
class_weights = [1]*num_classes
df['class_id'] = pd.factorize(df['medical_specialty'])[0]
df.head()

,medical_specialty,transcription,keywords,class_id
0,others,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller...",0
1,others,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh...",0
2,others,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart...",0
3,Cardiovascular / Pulmonary,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple...",1
4,Cardiovascular / Pulmonary,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo...",1


In [6]:
class_dict = df[['medical_specialty', 'class_id']].drop_duplicates().set_index('class_id')
print(class_dict)

dataset = pd.DataFrame({
    'text' : df['keywords']+df['transcription'],
    'labels' : df['class_id']
})

train_df, test_df = train_test_split(dataset, test_size=DS_SPLIT, random_state=42)

print(f"\nTrain Size: {len(train_df)}, Test Size: {len(test_df)}")
train_df.head()


                       medical_specialty
class_id                                
0                                 others
1             Cardiovascular / Pulmonary
2                                Urology
3                       General Medicine
4                                Surgery
5          SOAP / Chart / Progress Notes
6                              Radiology
7                             Orthopedic
8                Obstetrics / Gynecology
9                              Neurology
10                      Gastroenterology
11            Consult - History and Phy.

Train Size: 3118, Test Size: 780


,text,labels
2656,"neurosurgery, debridement of wound, shunt, cos...",0
3238,"general medicine, ears, nose, mouth, neck, res...",3
2114,"orthopedic, open reduction, internal fixation,...",7
1329,"soap / chart / progress notes, parathyroid hyp...",5
695,"surgery, hydrocele, hydrocele repair, hernia, ...",4


In [7]:
def eval_metrics(pred, actual):
    return {
        "accuracy": balanced_accuracy_score(actual,pred),
        "f1_weighted": f1_score(actual,pred,average="micro")
    }

In [8]:
cuda_available = torch.cuda.is_available()

model_args = ClassificationArgs(
    num_train_epochs=EPOCHS,
    learning_rate = LEARNING_RATE,
    reprocess_input_data= True,
    save_model_every_epoch=False,
    overwrite_output_dir= True
    )

model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=num_classes,
    weight=class_weights,
    use_cuda=cuda_available,
    args=model_args
    )

model.train_model(train_df, eval_df=test_df, custom_eval_function=eval_metrics)
model.save_model()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/390 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/390 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/390 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/390 [00:00<?, ?it/s]

Running Epoch 5 of 5:   0%|          | 0/390 [00:00<?, ?it/s]

In [9]:
result, _, _ = model.eval_model(test_df)
print(f"MCC: {result['mcc']}, Eval_Loss: {result['eval_loss']}")

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/8 [00:00<?, ?it/s]

MCC: 0.9877820934909955, Eval_Loss: 0.05131442143465392


In [10]:
result,output = model.predict(test_df['text'].values.tolist())

eval_result = eval_metrics(result, test_df['labels'])
print(f"Accuracy: {eval_result['accuracy']}, F1 Score: {eval_result['f1_weighted']}")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

Accuracy: 0.9826473283408336, F1 Score: 0.9897435897435898


In [11]:
samples = test_df.sample(5, random_state=42)
for _, row in samples.iterrows():
    transcription = row['text']
    true_class = class_dict.loc[row['labels'], 'medical_specialty']
    pred_label = model.predict([transcription])[0][0]
    pred_class = class_dict.loc[pred_label, 'medical_specialty']
    print(f"Transcription: {transcription}\nTrue Label: {true_class}, Predicted Label: {pred_class}\n")

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Transcription: soap / chart / progress notes, congestive heart failure, diabetes, hyperlipidemia, chronic renal insufficiency, arthritis, chronic medical conditions, heart, REASON FOR VISIT: , Followup evaluation and management of chronic medical conditions.,HISTORY OF PRESENT ILLNESS:,  The patient has been doing quite well since he was last seen.  He comes in today with his daughter.  He has had no symptoms of CAD or CHF.  He had followup with Dr. X and she thought he was doing quite well as well.  He has had no symptoms of hyperglycemia or hypoglycemia.  He has had no falls.  His right knee does pain him at times and he is using occasional doses of Tylenol for that.  He wonders whether he could use a knee brace to help him with that issue as well.  His spirits are good.  He has had no incontinence.  His memory is clear, as is his thinking.,MEDICATIONS:,1.  Bumex - 2 mg daily.,2.  Aspirin - 81 mg daily.,3.  Lisinopril - 40 mg daily.,4.  NPH insulin - 65 units in the morning and 25 un

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Transcription: surgery, claudication, extremity run off, angio suite, superficial femoral artery, popliteal, superficial, femoral, aortogram, artery, balloon, glidewire, angioplasty, stenosis, renal, PREPROCEDURE DIAGNOSIS:,  Left leg claudication.,POSTPROCEDURE DIAGNOSIS: ,  Left leg claudication.,OPERATION PERFORMED: , Aortogram with bilateral, segmental lower extremity run off.,ANESTHESIA: , Conscious sedation.,INDICATION FOR PROCEDURE:  ,The patient presents with lower extremity claudication.  She is a 68-year-old woman, who is very fearful of the aforementioned procedures.  Risks and benefits of the procedure were explained to her to include bleeding, infection, arterial trauma requiring surgery, access issues and recurrence.  She appears to understand and agrees to proceed.,DESCRIPTION OF PROCEDURE: , The patient was taken to the Angio Suite, placed in a supine position.  After adequate conscious sedation, both groins were prepped with Chloraseptic prep.  Cloth towels and paper d

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Transcription: pain management, nerve root block, 1% lidocaine, asa monitors, cervical selective nerve root block, fluoroscopy, iv sedation, nerve root, oblique, selective nerve root, carotid artery, foramen, neural foramina, edge of the foramen, selective nerve root block, cervical,CERVICAL SELECTIVE NERVE ROOT BLOCK,PREPROCEDURE PREPARATION:,  After being explained the risks and benefits of the procedure, the patient signed the standard informed consent form.  The patient was placed in the prone position and standard ASA monitors applied.  Intravenous access was established and IV sedation was used.  For further details of IV sedation and infusion, please refer to anesthesia notes.  The patient was able to respond appropriately throughout the procedure.  * Fluoroscopy was used to identify the appropriate anatomy.  The skin was prepped and draped in a sterile fashion and sterile technique was maintained throughout the procedure.,PROCEDURE DETAILS:,  The patient was laid supine.  Obliq

Rajapakse, T. C., Yates, A., & de Rijke, M. (2024). Simple Transformers: Open-source for all. In *Proceedings of the 2024 Annual International ACM SIGIR Conference on Research and Development in Information Retrieval in the Asia Pacific Region* (SIGIR-AP 2024, pp. 209–215). Association for Computing Machinery. [https://doi.org/10.1145/3673791.3698412](https://doi.org/10.1145/3673791.3698412)